In [2]:
using Globtim
using CSV
using DataFrames
include("../src/lib_func.jl")

# Load the dataframe from the CSV file
df_2d = CSV.read("../data/camel_3_d6.csv", DataFrame)

# Constants and Parameters
d = 3      # Degree 
const n, a, b = 4, 5, 1 
const scale_factor = a / b  # Scaling constant, C is appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .1 , 2 / 10  # Sampling parameters
const tol_l2 = 500               # Define the tolerance for the L2-norm
# const tol_l2 = 1e-0
# The objective function

f = camel_3_by_3 # Objective function

camel_3_by_3 (generic function with 1 method)

The `camel_3_by_3` function is the product `camel_3(x1,x2)*camel_3(x3,x4)`.
Maybe not the best example for 4d since the minimas are so shallow and the magnitude of the function blows up around the edges.

In [3]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, n, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        global d += 1
    end
end

current L2-norm: 15077.847551104931
Number of samples: 7
current L2-norm: 4367.137375343813
Number of samples: 9
current L2-norm: 2335.2243566828693
Number of samples: 12
current L2-norm: 590.2587682348413
Number of samples: 14
attained the desired L2-norm: 390.9501611610721


In [4]:
# Assuming x is already defined as @polyvar x[1:2]
loc = "inputs.ms"
# File path of the output file
file_path_output = "outputs.ms";

In [5]:
using DynamicPolynomials, DataFrames
ap = main_nd(n, d, poly_approx.coeffs)
@polyvar(x[1:n]) # Define polynomial ring 
# Expand the polynomial approximant to the standard monomial basis in the Lexicographic order w.r.t x
names = [x[i].name for i in 1:length(x)]
open(loc, "w") do file
    println(file, join(names, ", "))
    println(file, 0)
end
# Define the polynomial approximant 
PolynomialApproximant = sum(ap .* MonomialVector(x, 0:d))
for i in 1:n
    partial = differentiate(PolynomialApproximant, x[i])
    partial_str = replace(string(partial), "//" => "/")
    open(loc, "a") do file
        if i < n
            println(file, string(partial_str, ","))
        else
            println(file, partial_str)
        end
    end
end

Now we use Optim.jl package to initialize step 3 and converge to the local minimizers.
How do we know we have everything ? Use simple combinations of 

In [6]:
run(`msolve -v 1 -f inputs.ms -o outputs.ms`)


--------------- INPUT DATA ---------------
#variables                       4
#equations                       4
#invalid equations               0
field characteristic             0
homogeneous input?               0
signature-based computation      0
monomial order                 DRL
basis hash table resetting     OFF
linear algebra option            2
initial hash table size     131072 (2^17)
max pair selection             ALL
reduce gb                        1
#threads                         1
info level                       1
generate pbm files               0
------------------------------------------
Dimension of quotient: 1296
[1296, 146], Non trivial / Trivial = 11.27%
Density of non-trivial part 90.85%
Time spent to generate sequence (elapsed): 0.26 sec (1.88 Gops/sec)
Time spent to compute eliminating polynomial (elapsed): 0.04 sec
Elimination polynomial has degree 1296.

Starts multi-modular computations
{2}{4}{8}{16}{32}{64}{128}{256}{512}{1024}
<Step:2/0.06/0.29>{2048

ProcessFailedException: failed process: Process(`msolve -v 1 -f inputs.ms -o outputs.ms`, ProcessSignaled(9)) [0]


Construct a Dataframe with only the critical points of the approximant $w_d$ which fall into the $[-1, 1]^4$ domain and rescale them to the original domain. 

In [7]:
function average(X::Vector{Int})::Float64
    return sum(X) / length(X)
end

# Process the file and get the points
evaled = process_output_file(file_path_output)

# Parse the points into correct format
real_pts = []
for pts in evaled
    if typeof(pts) == Vector{Vector{Vector{BigInt}}}
        X = parse_point(pts)
    else
        X = average.(pts)
    end
    push!(real_pts, Float64.(X))
end

condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1 && -1 < point[3] < 1 && -1 < point[4] < 1 
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Colllect the critical points of the approximant 
h_x1 = scale_factor * Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_x2 = scale_factor * Float64[point[2] for point in filtered_points] 
h_x3 = scale_factor * Float64[point[3] for point in filtered_points] 
h_x4 = scale_factor * Float64[point[4] for point in filtered_points] 

height = map(p -> f(p), zip(h_x1, h_x2, h_x3, h_x4)) # Compute the height of the critical points
df = DataFrame(x1 = h_x1, x2 = h_x2, x3 = h_x3, x4 = h_x4, height = height); # Create a DataFrame

MethodError: MethodError: no method matching getindex(::Nothing, ::Int64)

Now we want to compute the distance between the critical points of the 2d function.

The true "exact" critical points of the 2d `Camel3` function are stored in `df_2d`. 




In [8]:
using LinearAlgebra

total_distance = Float64[]
for i in 1:nrow(df)
    distances_1 = [norm([df.x1[i], df.x2[i]] - [df_2d.x[j], df_2d.y[j]]) for j in 1:nrow(df_2d)]
    distances_2 = [norm([df.x3[i], df.x4[i]] - [df_2d.x[j], df_2d.y[j]]) for j in 1:nrow(df_2d)]
    # Compute distances to all points in df_2d
    min_value_1, min_index_1 = findmin(distances_1)
    min_value_2, min_index_2 = findmin(distances_2) 

    # Extract the corresponding points
    point_1 = [df.x1[i], df.x2[i], df.x3[i], df.x4[i]]
    point_2 = [df_2d.x[min_index_1], df_2d.y[min_index_1], df_2d.x[min_index_2], df_2d.y[min_index_2]]

    # Compute total distance
    tot_dist = norm(point_1 - point_2)
    push!(total_distance, tot_dist)

end
df.total_distance = total_distance
sorted_df = sort(df, :total_distance)


UndefVarError: UndefVarError: `df` not defined

In [9]:
using Optim
df.steps = zeros(nrow(df))
df.converged = zeros(nrow(df))

for i in 1:nrow(df)
    # println("Optimizing for point $i")
    x0 = [df.x1[i], df.x2[i], df.x3[i], df.x4[i]]
    #  + 0.2 * randn(Float64, 4)
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=false))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    steps = res.iterations
    converged = Optim.converged(res)
    distance = norm(x0 - minimizer)
    # df.dist_to_loc_min[i] = distance
    df.steps[i] = steps
    df.converged[i] = converged
    # summary(res)
end
df

UndefVarError: UndefVarError: `df` not defined